In [1]:
%pip install pandas==1.5.1
%pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 861 kB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 14.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from torchvision.transforms.functional import hflip
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from matplotlib import image as mpimg
import re
from PIL import Image
import pandas as pd
import datetime
from torchmetrics import TweedieDevianceScore

import math
from torch import default_generator, randperm
from torch._utils import _accumulate
from torch.utils.data.dataset import Subset

from sklearn.metrics import fbeta_score, make_scorer, mean_poisson_deviance
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer



In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

PATH='/content/drive/MyDrive/MLproject/'

with open(PATH+"listings.pkl", "rb") as fh:
    data = pickle.load(fh)
with open(PATH+"test_listings_image.pkl", "rb") as fh:
    data_test = pickle.load(fh)

cuda


# Dataset

In [51]:
data.columns

Index(['Are Pets Allowed', 'Category Idx', 'Day of Advertisement Created',
       'Day of Date Available From', 'Demand', 'Floor', 'Geo Canton',
       'Geo City', 'Geo Zip', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Jahr', 'Leerwohnungsziffer',
       'Listing Description', 'Listing Title', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Property Reference Id',
       'Size M2 Normalized', 'Subcategory En Idx', 'Year Built',
       'Year Lastrenovated'],
      dtype='object')

In [52]:
index = data['Property Reference Id']

In [53]:
features = data[['Are Pets Allowed', 'Category Idx', 'Day of Advertisement Created',
       'Day of Date Available From', 'Floor', 'Geo Canton',
       'Geo City', 'Geo Zip', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Jahr', 'Leerwohnungsziffer',
       'Listing Description', 'Listing Title', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Property Reference Id',
       'Size M2 Normalized', 'Subcategory En Idx', 'Year Built',
       'Year Lastrenovated']]

label = data['Demand']


# 1. preprocessing

In [ ]:
seed=np.random.randint(100)
DICT_GEO_CANTON={'ZH':0, 'VD':1, 'BE':2, 'AG':3, 'FR':4, 'SG':5, 'LU':6, 'SO':7, 'SH':8, 'GE':9, 'VS':10,
       'SZ':11, 'BS':12, 'ZG':13, 'BL':14, 'NE':15, 'TI':16, 'GR':17, 'TG':18, 'AR':19, 'UR':20, 'GL':21,
       'NW':22, 'JU':22}

DICT_SUBCATEGORY={'Apartment':0, 'Duplex':1, 'Row house':2, 'Roof flat':3, 'Attic flat':4,
       'Single house':5, 'Single Room':6, 'Studio':7, 'Furnished dwelling':8,
       'Loft':9, 'Mansard':10, 'Farm house':11}


#identifying the boolean variables
bool_features=['Are Pets Allowed', 'Has Balcony', 'Has Cabletv',
               'Has Elevator', 'Has Fireplace', 'Has Garage',
               'Has Parking', 'Is New Construction', 'Is New Construction Potential', 
               'Is Tenant2Tenant', 'Is Wheelchairaccessible']
print(bool_features)
data_bool=data[bool_features]


#The ordinal features are:
ordinal_features=['Floor','Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 
       'Size M2 Normalized', 'Year Built',
       'Year Lastrenovated']

categorical_features={'Geo Canton':DICT_GEO_CANTON,'Subcategory En Idx':DICT_SUBCATEGORY}

#Which leaves us with 'Property Reference Id ',Listing Description' and 'Listing Title' which we will not use for regression purpose
data=data.drop(['Property Reference Id','Listing Description','Listing Title'],axis=1)

null_count=pd.concat([data.isnull().sum().rename('NaN train set'),data_test.isnull().sum().rename('NaN test set')],axis=1)

iter_imputer = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10, random_state=0),random_state=42,max_iter=20)

temp=data.copy()

iter_imputed = iter_imputer.fit_transform(data[ordinal_features])

temp.loc[:,ordinal_features]=iter_imputed




['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction', 'Is New Construction Potential', 'Is Tenant2Tenant', 'Is Wheelchairaccessible']


/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
<ipython-input-10-cca028cf8e98>:41: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  temp.loc[:,ordinal_features]=iter_imputed


In [ ]:

#Deal with 'Day of Advertisement Created' and 'Day of Date Available From'
def compute_diff_date(X: pd.DataFrame, y=None):
    X_copy=X.copy()
    threshold=datetime.datetime(2022,9,1)
    X_copy['Days Between September and Created']=pd.to_datetime(X_copy['Day of Advertisement Created']).apply(lambda x:(threshold-x).days)
    X_copy['Days Between September and Available']=pd.to_datetime(X_copy['Day of Date Available From']).apply(lambda x:(threshold-x).days)                                                           
    X_copy.drop(['Day of Advertisement Created','Day of Date Available From'],axis=1,inplace=True)
    return X_copy

diff_date_transformer = FunctionTransformer(compute_diff_date)

print('Dealing with date')
display(diff_date_transformer.transform(data))

Dealing with date


,Are Pets Allowed,Category Idx,Demand,Floor,Geo Canton,Geo City,Geo Zip,Has Balcony,Has Cabletv,Has Elevator,...,Price Extra Normalized,Price Gross Normalized,Price M2 Normalized,Price Net Normalized,Size M2 Normalized,Subcategory En Idx,Year Built,Year Lastrenovated,Days Between September and Created,Days Between September and Available
0,NaN,APPT,25.0,0.0,ZH,Dübendorf,8600,NaN,NaN,NaN,...,550.0,4460.0,19.0,3910.0,232.0,Apartment,2016.0,2016.0,602,549.0
1,NaN,APPT,2.0,1.0,VD,Le Mont-sur-Lausanne,1052,NaN,NaN,NaN,...,120.0,1345.0,30.0,1225.0,45.0,Apartment,NaN,NaN,1018,914.0
2,False,APPT,0.0,1.0,BE,Langenthal,4900,False,False,False,...,250.0,1570.0,15.0,1320.0,103.0,Apartment,NaN,NaN,1996,NaN
3,True,APPT,4.0,1.0,AG,Fislisbach,5442,NaN,True,NaN,...,270.0,1480.0,17.0,1210.0,86.0,Apartment,1985.0,1985.0,421,488.0
4,False,APPT,3.0,0.0,FR,Bulle,1630,False,False,False,...,300.0,1490.0,19.0,1190.0,79.0,Apartment,NaN,NaN,1763,1857.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,APPT,2.0,2.0,SG,St. Gallen,9000,False,False,False,...,170.0,1350.0,15.0,1180.0,90.0,Apartment,NaN,NaN,1701,1614.0
19996,True,APPT,1.0,2.0,VD,Vevey,1800,NaN,True,True,...,190.0,1940.0,23.0,1750.0,84.0,Apartment,1968.0,1968.0,302,212.0
19997,NaN,APPT,2.0,5.0,GE,Lancy,1212,NaN,NaN,NaN,...,275.0,2175.0,32.0,1900.0,68.0,Apartment,NaN,NaN,321,442.0
19998,NaN,APPT,2.0,7.0,VD,Nyon,1260,NaN,NaN,NaN,...,240.0,3540.0,37.0,3300.0,95.0,Apartment,NaN,NaN,700,488.0


In [ ]:
other_categorical_features=['Category Idx','Geo City','Geo Zip']
data=data.drop(other_categorical_features,axis=1)

In [ ]:
categorical_features={'Geo Canton':DICT_GEO_CANTON,'Subcategory En Idx':DICT_SUBCATEGORY}


In [ ]:
def fill_nan(X: pd.DataFrame,bool_features,categorical_features,ordinal_features,bool_fill=-99999, y=None):
    df=X.copy() #We don't want to modify the dataset
    
    #Dealing with categorical values
    for bool_feature in bool_features:
        df.loc[:,bool_feature]=df.loc[:,bool_feature].fillna(bool_fill)
        df.loc[df[bool_feature] == False, bool_feature] = 0
        df.loc[data[bool_feature] == True, bool_feature] = 1
    
    #Dealing with ordinal values
    for ordinal_feature in ordinal_features:
        df[ordinal_feature].fillna(df[ordinal_feature].mean(),inplace=True)
        
    #Dealing with categorical features
    for categorical_feature in categorical_features.keys():
        dict_replace=categorical_features[categorical_feature]
        df[categorical_feature]=df[categorical_feature].replace(dict_replace)
        df[categorical_feature].fillna(len(dict_replace)+1,inplace=True)
        #df[categorical_feature].fillna(bool_fill,inplace=True)
    return df

na_fill_estimator = FunctionTransformer(fill_nan,kw_args={'bool_features':bool_features,
                                                          'categorical_features':categorical_features,
                                'ordinal_features':ordinal_features+['Days Between September and Created',
                                                                     'Days Between September and Available']})


features_to_encode=bool_features
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough"
                        )
print('One Hot Encoded data:')
print(col_trans.fit_transform(na_fill_estimator.transform(diff_date_transformer.transform(data))))
print(col_trans.fit_transform(na_fill_estimator.transform(diff_date_transformer.transform(data))).shape)



One Hot Encoded data:
[[1.0 0.0 0.0 ... 2016.0 602 549.0]
 [1.0 0.0 0.0 ... 1991.7910962715637 1018 914.0]
 [0.0 1.0 0.0 ... 1991.7910962715637 1996 1028.099604743083]
 ...
 [1.0 0.0 0.0 ... 1991.7910962715637 321 442.0]
 [1.0 0.0 0.0 ... 1991.7910962715637 700 488.0]
 [0.0 1.0 0.0 ... 1991.7910962715637 1939 1826.0]]
(20000, 51)


# 2. Real preprocessing

In [54]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)


def preprocess(data):
  binary_features = ['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
                  'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
                  'Is New Construction Potential', 'Is Tenant2Tenant',
                  'Is Wheelchairaccessible']
  for column in binary_features:
    data.loc[data[column] == False, column] = 0
    data.loc[data[column] == True, column] = 1

  data = pd.get_dummies(data, prefix=['Geo'], columns=['Geo Canton'])

  data['Year Lastrenovated'].fillna(0, inplace = True)

  threshold=datetime.datetime(2022,9,1)
  data['Days Between September and Created']=pd.to_datetime(data['Day of Advertisement Created']).apply(lambda x:(threshold-x).days)
  data['Days Between September and Available']=pd.to_datetime(data['Day of Date Available From']).apply(lambda x:(threshold-x).days)                                                           
  # data.drop(['Day of Advertisement Created','Day of Date Available From'],axis=1,inplace=True)

  data['Leerwohnungsziffer'] = pd.to_numeric(data['Leerwohnungsziffer'])
  data['Jahr'] = pd.to_numeric(data['Jahr'])

  output_cols = list(data['Demand'])

  input_cols = data[['Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized', 
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr',
       'Geo_AG', 'Geo_AR', 'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 
       'Geo_GL', 'Geo_GR', 'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 
       'Geo_SH', 'Geo_SO', 'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG', 'Geo_ZH']]

  # Normalization of feature columns

  input_cols_norm = mean_norm(input_cols)
  input_cols_norm.fillna(input_cols_norm.median(), inplace=True)
  input_cols_norm.sample(10)

  input_cols_norm['Demand'] = data['Demand']
  input_cols_norm['Day of Advertisement Created'] = data['Day of Advertisement Created']
  input_cols_norm['Day of Date Available From'] = data['Day of Date Available From']

  first_column = input_cols_norm.pop('Day of Advertisement Created')
  input_cols_norm.insert(0, 'Day of Advertisement Created', first_column)
  second_column = input_cols_norm.pop('Day of Date Available From')
  input_cols_norm.insert(0, 'Day of Date Available From', second_column)

  input_cols_norm['Property Reference Id'] = index
  
  return data, input_cols_norm


data, input_cols_norm = preprocess(data)


In [76]:
input_cols_norm.head(5)

,Day of Date Available From,Day of Advertisement Created,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Property Reference Id
0,"March 1, 2021","January 7, 2021",-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,25.0,0002.03.7648
1,"March 1, 2020","November 18, 2019",-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,2.0,9680.61.2071
2,NaN,"March 15, 2017",-0.832208,-0.602958,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,0.0,7204.61.8658
3,"May 1, 2021","July 7, 2021",1.201510,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,4.0,7410.50.5987
4,"August 1, 2017","November 3, 2017",-0.832208,-1.067466,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,3.0,9270.61.7822


In [77]:
data.head(5)

,Are Pets Allowed,Category Idx,Day of Advertisement Created,Day of Date Available From,Demand,Floor,Geo City,Geo Zip,Has Balcony,Has Cabletv,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Days Between September and Created,Days Between September and Available
0,NaN,APPT,"January 7, 2021","March 1, 2021",25.0,0.0,Dübendorf,8600,NaN,NaN,...,0,0,0,0,0,0,0,1,602,549.0
1,NaN,APPT,"November 18, 2019","March 1, 2020",2.0,1.0,Le Mont-sur-Lausanne,1052,NaN,NaN,...,0,0,0,0,1,0,0,0,1018,914.0
2,0,APPT,"March 15, 2017",NaN,0.0,1.0,Langenthal,4900,0,0,...,0,0,0,0,0,0,0,0,1996,NaN
3,1,APPT,"July 7, 2021","May 1, 2021",4.0,1.0,Fislisbach,5442,NaN,1,...,0,0,0,0,0,0,0,0,421,488.0
4,0,APPT,"November 3, 2017","August 1, 2017",3.0,0.0,Bulle,1630,0,0,...,0,0,0,0,0,0,0,0,1763,1857.0


## 1st version for data processing (no need anymore)

In [17]:
binary_features = ['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
                  'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
                  'Is New Construction Potential', 'Is Tenant2Tenant',
                  'Is Wheelchairaccessible']
for column in binary_features:
  data.loc[data[column] == False, column] = 0
  data.loc[data[column] == True, column] = 1

data = pd.get_dummies(data, prefix=['Geo'], columns=['Geo Canton'])

data['Year Lastrenovated'].fillna(0, inplace = True)

threshold=datetime.datetime(2022,9,1)
data['Days Between September and Created']=pd.to_datetime(data['Day of Advertisement Created']).apply(lambda x:(threshold-x).days)
data['Days Between September and Available']=pd.to_datetime(data['Day of Date Available From']).apply(lambda x:(threshold-x).days)                                                           
# data.drop(['Day of Advertisement Created','Day of Date Available From'],axis=1,inplace=True)


data['Leerwohnungsziffer'] = pd.to_numeric(data['Leerwohnungsziffer'])
data['Jahr'] = pd.to_numeric(data['Jahr'])


KeyError: ignored

In [18]:
output_cols = list(data['Demand'])

input_cols = data[['Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized', 
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr',
       'Geo_AG', 'Geo_AR', 'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 
       'Geo_GL', 'Geo_GR', 'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 
       'Geo_SH', 'Geo_SO', 'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG', 'Geo_ZH']]

# Normalization of feature columns

def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

input_cols_norm = mean_norm(input_cols)
input_cols_norm.fillna(input_cols_norm.median(), inplace=True)
input_cols_norm.sample(10)

input_cols_norm['Demand'] = data['Demand']
input_cols_norm['Day of Advertisement Created'] = data['Day of Advertisement Created']
input_cols_norm['Day of Date Available From'] = data['Day of Date Available From']

first_column = input_cols_norm.pop('Day of Advertisement Created')
input_cols_norm.insert(0, 'Day of Advertisement Created', first_column)
second_column = input_cols_norm.pop('Day of Date Available From')
input_cols_norm.insert(0, 'Day of Date Available From', second_column)


In [19]:
input_cols_norm['Property Reference Id'] = index
input_cols_norm

,Day of Date Available From,Day of Advertisement Created,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Property Reference Id
0,"March 1, 2021","January 7, 2021",-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,25.0,0002.03.7648
1,"March 1, 2020","November 18, 2019",-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,2.0,9680.61.2071
2,NaN,"March 15, 2017",-0.832208,-0.602958,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,0.0,7204.61.8658
3,"May 1, 2021","July 7, 2021",1.201510,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,4.0,7410.50.5987
4,"August 1, 2017","November 3, 2017",-0.832208,-1.067466,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,3.0,9270.61.7822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,"April 1, 2018","January 4, 2018",-0.832208,-0.138449,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,4458.50.0065
19996,"February 1, 2022","November 3, 2021",1.201510,-0.138449,-0.200172,0.758779,0.798902,-0.156072,0.899897,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,1.0,4481.61.5904
19997,"June 16, 2021","October 15, 2021",-0.832208,1.255078,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,0221.61.7685
19998,"May 1, 2021","October 1, 2020",-0.832208,2.184095,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,2.0,8508.50.4399


# Merge with image dataset



In [56]:
# images_data = pd.read_csv(PATH + "images_vectorized.csv")
images_data = pd.read_csv(PATH + "images_vectorized_2.csv")

In [58]:
def merge_dataset(tabular_data, image_data):

  images_id = images_data['Property Reference Id'].unique()
  print("image_id:", images_id)
  print("len(images_id):", len(images_id))

  tabular_data = tabular_data[tabular_data['Property Reference Id'].isin(images_id)]

  images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')


  images_data.drop('Demand', inplace=True, axis=1)
  data_joined = images_data.merge(input_cols_norm_reduced, on='Property Reference Id', how='left')

  return data_joined



data_joined = merge_dataset(input_cols_norm, images_data)


image_id: ['0002.61.6915' '0012.61.9682' '0043.61.2130' '0051.06.1645'
 '0053.61.6292' '0092.22.2130' '0092.26.3101' '0129.26.1770'
 '0129.61.9412' '0200.26.4088' '0214.61.8006' '0221.61.7685'
 '0254.50.1151' '0286.26.1334' '0286.50.7822' '0292.61.3115'
 '0294.61.7107' '0300.61.6054' '0300.61.9894' '0311.50.7705'
 '0311.61.1334' '0356.50.7822' '0359.61.2130' '0480.06.4593'
 '0618.06.0031' '0662.26.4741' '0662.26.9662' '0662.50.9670'
 '0668.61.0065' '0759.61.3115' '0759.61.6450' '0789.34.5829'
 '0789.34.7688' '0836.50.1151' '0837.50.0065' '0862.06.3115'
 '0908.26.5664' '0913.22.3690' '0947.61.9754' '0947.92.2532'
 '0960.61.2321' '1036.50.0065' '1036.50.6369' '1036.50.8115'
 '1036.61.6915' '1036.61.7416' '1036.61.7543' '1038.06.5834'
 '1038.50.0065' '1073.26.9682' '1076.06.4741' '1076.06.9424'
 '1076.06.9662' '1076.50.2130' '1107.61.9424' '1120.26.1351'
 '1137.26.1334' '1137.61.5386' '1232.61.9935' '1297.50.0031'
 '1297.50.7822' '1297.50.9894' '1297.61.5386' '1297.61.9754'
 '1308.50.5240

<ipython-input-58-b583ff644b11>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')


In [59]:
# data_joined.drop('Property Reference Id', inplace=True, axis=1)
data_joined['Demand']

0        2.0
1        2.0
2        2.0
3       17.0
4       17.0
        ... 
8476     5.0
8477     5.0
8478     5.0
8479     5.0
8480     5.0
Name: Demand, Length: 8481, dtype: float64

In [60]:
data_joined.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', 'Day of Date Available From',
       'Day of Advertisement Created', 'Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG'

In [61]:
data_joined

,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8477,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8478,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8479,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0


## merge version 1

In [46]:
# images_data = pd.read_csv(PATH + "images_vectorized.csv")
images_data = pd.read_csv(PATH + "images_vectorized_2.csv")

images_id = images_data['Property Reference Id'].unique()
print(images_id)
print(len(images_id))


['0002.61.6915' '0012.61.9682' '0043.61.2130' '0051.06.1645'
 '0053.61.6292' '0092.22.2130' '0092.26.3101' '0129.26.1770'
 '0129.61.9412' '0200.26.4088' '0214.61.8006' '0221.61.7685'
 '0254.50.1151' '0286.26.1334' '0286.50.7822' '0292.61.3115'
 '0294.61.7107' '0300.61.6054' '0300.61.9894' '0311.50.7705'
 '0311.61.1334' '0356.50.7822' '0359.61.2130' '0480.06.4593'
 '0618.06.0031' '0662.26.4741' '0662.26.9662' '0662.50.9670'
 '0668.61.0065' '0759.61.3115' '0759.61.6450' '0789.34.5829'
 '0789.34.7688' '0836.50.1151' '0837.50.0065' '0862.06.3115'
 '0908.26.5664' '0913.22.3690' '0947.61.9754' '0947.92.2532'
 '0960.61.2321' '1036.50.0065' '1036.50.6369' '1036.50.8115'
 '1036.61.6915' '1036.61.7416' '1036.61.7543' '1038.06.5834'
 '1038.50.0065' '1073.26.9682' '1076.06.4741' '1076.06.9424'
 '1076.06.9662' '1076.50.2130' '1107.61.9424' '1120.26.1351'
 '1137.26.1334' '1137.61.5386' '1232.61.9935' '1297.50.0031'
 '1297.50.7822' '1297.50.9894' '1297.61.5386' '1297.61.9754'
 '1308.50.5240' '1308.50

In [23]:
input_cols_norm_reduced = input_cols_norm[input_cols_norm['Property Reference Id'].isin(images_id)]
input_cols_norm_reduced

,Day of Date Available From,Day of Advertisement Created,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Property Reference Id
51,"September 1, 2020","July 3, 2020",-0.832208,0.326060,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,19.0,7415.50.2130
56,"September 1, 2018","August 6, 2018",-0.832208,-1.067466,-0.200172,-1.317803,0.798902,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,9.525680,-0.104730,-0.723488,0.0,5469.26.4284
140,"March 1, 2019","October 24, 2018",-0.832208,-0.138449,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,0.0,7276.22.0065
145,"March 1, 2022","November 30, 2021",-0.832208,0.326060,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.104730,-0.723488,8.0,5361.50.7934
177,"November 1, 2018","August 7, 2018",1.201510,-0.602958,-0.200172,0.758779,-1.251618,-0.156072,-1.111143,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,0.0,4175.61.4362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19888,"September 1, 2017","June 2, 2017",-0.832208,0.326060,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,19.585612,-0.259622,-0.104974,-0.104730,-0.723488,0.0,8396.61.5664
19932,"May 1, 2021","April 15, 2021",-0.832208,-0.138449,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,2.0,0092.26.3101
19941,"April 1, 2020","March 4, 2020",-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,1.382123,16.0,5738.50.4362
19976,"April 1, 2019","January 17, 2019",-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,9.547845,-0.723488,13.0,8994.68.7822


In [24]:
# images_data = pd.read_csv(PATH + "images_vectorized.csv")
images_data = pd.read_csv(PATH + "images_vectorized_2.csv")

images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
input_cols_norm_reduced['Property Reference Id'] = input_cols_norm_reduced['Property Reference Id'].astype('category')
images_data

<ipython-input-24-3d3e7fa5b4a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_cols_norm_reduced['Property Reference Id'] = input_cols_norm_reduced['Property Reference Id'].astype('category')


,Property Reference Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,6.887189,-9.850289,-0.950801,-2.359023,-7.314687,9.682235,-2.985931,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,6.779986,-9.572109,-0.092465,-2.760837,-7.171674,9.206720,-2.622147,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,7.241723,-10.057664,0.637905,-3.071800,-7.520823,9.508305,-3.589651,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,7.880892,-12.079361,0.409625,-3.946983,-8.715199,11.561431,-4.128768,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,6.727478,-9.977320,-1.542024,-1.799264,-7.152509,10.005502,-2.396945,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5838,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,10.430278,-15.639960,2.439432,-6.165233,-11.678176,13.718158,-2.538267,5.0
5839,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,9.185239,-13.673468,0.599522,-4.164209,-10.169299,12.084097,-1.126678,5.0
5840,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,8.530967,-13.685159,3.585144,-6.815229,-10.193208,12.663684,7.011076,5.0
5841,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,7.720353,-13.286383,-1.674972,-4.154854,-8.953053,13.140576,2.727453,5.0


In [25]:
images_data.drop('Demand', inplace=True, axis=1)
data_joined = images_data.merge(input_cols_norm_reduced, on='Property Reference Id', how='left')
data_joined

,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8477,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8478,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8479,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0


# We split now to facilitate exposure of results at the end (with Reference ID)

In [62]:
import random
import math
unique_ids=data_joined['Property Reference Id'].unique()
# We define the ID we use for training and those we use for testing
ids_train=np.array(random.sample(set(unique_ids), math.floor(0.8*len(unique_ids))))
ids_test=np.array([x for x in unique_ids if x not in ids_train])
print(len(unique_ids),len(ids_train),len(ids_test))

#We define the train dataset and the test dataset
data_joined_train=data_joined[data_joined['Property Reference Id'].isin(ids_train)]
data_joined_test=data_joined[data_joined['Property Reference Id'].isin(ids_test)]
print(len(data_joined),len(data_joined_train),len(data_joined_test))


686 548 138
8481 6827 1654


# Data to tensors

In [63]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.autograd import Variable
import torchvision.models as models

In [64]:
# label = data_joined['Demand']

# output_cols = list(data_joined['Demand'])

# input_cols = data_joined[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
#        '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
#        'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
#        'Has Parking', 'Is New Construction', 'Is New Construction Potential',
#        'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
#        'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
#        'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
#        'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
#        'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
#        'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
#        'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
#        'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
#        'Geo_ZH']]


label = data_joined_train['Demand']
output_cols = list(label)

input_cols = data_joined_train[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
       'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
       'Has Parking', 'Is New Construction', 'Is New Construction Potential',
       'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       'Geo_ZH']]

In [65]:
input_cols

,0,1,2,3,4,5,6,7,8,9,...,Geo_SH,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH
0,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,6.887189,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
1,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,6.779986,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
2,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,7.241723,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
3,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,7.880892,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488
4,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,6.727478,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,10.430278,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
8477,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,9.185239,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
8478,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,8.530967,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
8479,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,7.720353,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488


In [66]:
#TRAIN PART
inputs_array, targets_array1 = np.array(input_cols.to_numpy()), label.to_numpy()
targets_array=[]
#We add brakets for tensor structure... (can be done differently)
for elt in targets_array1:
    targets_array.append([elt])
targets_array = np.array(targets_array)

print(type(inputs_array))
print(type(targets_array))
print(inputs_array)
print(targets_array)

inputs = torch.from_numpy(inputs_array).to(torch.float32)
targets = torch.from_numpy(targets_array).to(torch.float32)
dataset = TensorDataset(inputs, targets)

print(dataset)


#TEST PART --------------------------------------------------------------------------------------------------------------------------

label_test = data_joined_test['Demand']
output_cols_test = list(label_test)

input_cols_test = data_joined_test[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
       'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
       'Has Parking', 'Is New Construction', 'Is New Construction Potential',
       'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       'Geo_ZH']]

inputs_array_test, targets_array1_test = np.array(input_cols_test.to_numpy()), label_test.to_numpy()
targets_array_test=[]
#We add brakets for tensor structure... (can be done differently)
for elt in targets_array1_test:
    targets_array_test.append([elt])
targets_array_test = np.array(targets_array_test)

print(type(inputs_array_test))
print(type(targets_array_test))
print(inputs_array_test)
print(targets_array_test)

inputs_test = torch.from_numpy(inputs_array_test).to(torch.float32)
targets_test = torch.from_numpy(targets_array_test).to(torch.float32)
dataset_test = TensorDataset(inputs_test, targets_test)

print(dataset_test)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-0.22183222  6.0242653   5.953249   ... -0.10497413 -0.10473044
   1.3821235 ]
 [ 0.60948485  6.2427998   5.7832255  ... -0.10497413 -0.10473044
   1.3821235 ]
 [ 1.3985655   6.613178    6.1452627  ... -0.10497413 -0.10473044
   1.3821235 ]
 ...
 [ 1.9538296   9.546097    7.4726825  ... -0.10497413 -0.10473044
  -0.72348817]
 [-1.8644348   7.3701596   6.550863   ... -0.10497413 -0.10473044
  -0.72348817]
 [ 0.11926779  6.666862    5.9892855  ... -0.10497413 -0.10473044
  -0.72348817]]
[[2.]
 [2.]
 [2.]
 ...
 [5.]
 [5.]
 [5.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[ 1.0867765   5.9109826   5.50825    ... -0.10497413 -0.10473044
  -0.72348817]
 [ 1.0867765   5.9109826   5.50825    ... -0.10497413 -0.10473044
  -0.72348817]
 [ 0.74761325  8.044648    6.853884   ... -0.10497413 -0.10473044
  -0.72348817]
 ...
 [-0.6344227   6.5021834   6.0300403  ... -0.10497413 -0.10473044
  -0.72348817]
 [ 0.2644492   5.2437377   5.1685996  ...

In [67]:
# VAL_PERCENT = 0.2
# NUM_ROWS = int(len(data_joined))
# val_size = int(NUM_ROWS * VAL_PERCENT)
# train_size = NUM_ROWS - val_size

# train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_ds = dataset
val_ds = dataset_test

print(len(train_ds), len(val_ds))

6827 1654


In [68]:
BATCH_SIZE = 128

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

for xb, yb, in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[-0.0421,  5.4728,  5.4674,  ..., -0.1050, -0.1047, -0.7235],
        [ 2.1562, 10.0712,  8.3192,  ..., -0.1050, -0.1047, -0.7235],
        [ 1.3466,  7.7821,  6.7120,  ...,  9.5257, -0.1047, -0.7235],
        ...,
        [-1.1796,  3.9899,  4.4664,  ..., -0.1050, -0.1047, -0.7235],
        [ 1.3782,  4.8351,  4.6566,  ..., -0.1050, -0.1047, -0.7235],
        [-1.1058,  3.9129,  4.4137,  ..., -0.1050, -0.1047, -0.7235]])
targets: tensor([[ 0.],
        [ 2.],
        [ 3.],
        [ 1.],
        [26.],
        [ 4.],
        [ 0.],
        [ 0.],
        [17.],
        [ 3.],
        [ 0.],
        [ 9.],
        [ 5.],
        [19.],
        [ 0.],
        [ 2.],
        [ 4.],
        [ 0.],
        [ 2.],
        [ 0.],
        [14.],
        [ 0.],
        [ 5.],
        [ 2.],
        [ 4.],
        [ 0.],
        [56.],
        [ 2.],
        [ 3.],
        [10.],
        [ 2.],
        [ 0.],
        [ 2.],
        [ 4.],
        [ 3.],
        [56.],
        [

# Model


In [69]:
#Creation of linear regression model

in_cols = input_cols.columns
out_cols = ['Demand']

input_size = len(in_cols)
output_size = len(out_cols)

class PredictModel(nn.Module):


    def __init__(self):
        # super().__init__()
        # self.linear = nn.Linear(input_size, output_size)
        super(PredictModel, self).__init__()
        self.layer_1 = nn.Linear(input_size, 32)
        self.layer_2 = nn.Linear(32, 64)
        self.layer_3 = nn.Linear(64, 128)
        # self.layer_4 = nn.Linear(64, 128)
        # self.layer_5 = nn.Linear(128, 64)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, output_size)
        
        self.relu = nn.ReLU()
    
    def forward(self, xb):
        # out = self.linear(xb)
        x = self.relu(self.layer_1(xb))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.relu(self.layer_4(x))
        x = self.relu(self.layer_5(x))
        # x = self.relu(self.layer_6(x))
        # x = self.relu(self.layer_7(x))
        x = self.layer_out(x)
        return x


    def training_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        if (epoch+1) % 20 == 0 or epoch == num_epochs - 1:
            print("Epoch[{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [70]:
#Defining our model 
model = PredictModel()
list(model.parameters())

#Training loop
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history=[]
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

result = evaluate(model, val_loader)
print(result)




{'val_loss': 4.688284873962402}


In [71]:
epochs = 300
lr = 0.001
history = fit(epochs, lr, model, train_loader, val_loader)

Epoch[20], val_loss: 4.1760
Epoch[40], val_loss: 4.1348
Epoch[60], val_loss: 4.0624
Epoch[80], val_loss: 3.8588
Epoch[100], val_loss: 3.6757
Epoch[120], val_loss: 3.5376
Epoch[140], val_loss: 3.5069
Epoch[160], val_loss: 3.3541
Epoch[180], val_loss: 3.6258
Epoch[200], val_loss: 3.6316
Epoch[220], val_loss: 3.5200
Epoch[240], val_loss: 3.5085
Epoch[260], val_loss: 3.4520
Epoch[280], val_loss: 3.7869
Epoch[300], val_loss: 3.4997


In [72]:
# We try to test our model on a random value of the test set

def predict_single(input, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)         
    prediction = predictions[0].detach()
    return prediction

input, target = val_ds[50]
prediction = predict_single(input, model)

print("Input:", input)
print("Target:", target)
print("Prediction:", prediction)
print()


# Return the entire predictions (Tensor Y_label)

def predict_all(val_ds, model):
    predictions = []
    targets = []
    for i in range(len(val_ds)):
       input, target = val_ds[i]
       prediction = predict_single(input, model)
       predictions.append(prediction.numpy()[0])
       targets.append(target)
    return predictions, targets

predictions, targets = predict_all(val_ds, model)

#On arrondi à l'entier le plus proche pour effectuer notre prédiction et on remplace les valeurs négatives

predictions = [max(10**(-0.05),k) for k in predictions]



Input: tensor([-1.6406,  4.0019,  4.6085, -4.7336,  0.8047,  0.6370, -0.6352,  1.7242,
         1.5807,  5.5719, -7.5750, -1.7852, -0.6885, -5.4793,  7.1512,  0.1325,
        -0.8322, -0.1384, -0.2002, -1.3178, -1.2516, -0.1561, -1.1111, -0.9166,
        -0.0794, -0.1257, -0.1090, -0.0535, -0.7679, -1.2049, -1.2049, -1.4841,
         0.6013, -1.4196, -1.3831, -0.1515, -1.3027, -0.7682, -0.9030,  0.0772,
        -1.3000, -0.3558, -0.0490, -0.2556, -0.2370, -0.1906,  6.2513, -0.2108,
        -0.0436, -0.0723, -0.0122, -0.2217, -0.1367, -0.0464, -0.2688, -0.0997,
        -0.1968, -0.0971, -0.1323, -0.1477, -0.0511, -0.2596, -0.1050, -0.1047,
        -0.7235])
Target: tensor([14.])
Prediction: tensor([1.1002])



In [83]:
print(predictions)

[0.8912509381337456, 3.0695283, 0.8912509381337456, 2.5105228, 0.8912509381337456, 0.9792241, 0.9362416, 0.8912509381337456, 0.8912509381337456, 3.215195, 0.8912509381337456, 3.0987277, 0.8912509381337456, 3.2575688, 0.8912509381337456, 3.322935, 0.8912509381337456, 3.1820562, 4.0517817, 0.8912509381337456, 4.2953806, 0.8912509381337456, 4.8679924, 4.686035, 3.858879, 4.1907544, 3.9270544, 0.9257528, 0.9669641, 1.679804, 1.5676266, 1.59382, 1.4589137, 1.5054209, 1.5177473, 1.383075, 2.028432, 1.85527, 1.6453567, 1.4892995, 1.4755293, 1.5352771, 1.5160413, 1.5497049, 1.5541018, 1.5081553, 1.0830669, 1.1225759, 1.1080743, 1.1147336, 1.1002134, 1.1074162, 1.2484486, 1.0265936, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 2.7459671, 3.3404505, 2.6485043, 3.2706823, 2.871703, 3.294888, 2.8989508, 3.4467304, 2.7924187, 3.1061685, 2.8130112, 3.1590366, 2.9751782, 3.360797, 3.0746894, 3.600231, 2.8553529, 3.2443752, 0.8912509381337456, 4.5009904, 4.619779, 4.

In [74]:
#Computation of the accurcy of our model on test set
EPSILON = 2

def accuracy(val_ds, model, EPSILON):
    corrects = 0
    for i in range(len(val_ds)):
        input, target = val_ds[i]
        prediction = predict_single(input, model)
        prediction = np.round(prediction)
        if prediction <= target + EPSILON and prediction >= target - EPSILON:
            corrects+=1
    accuracy = corrects/len(val_ds)
    print("Accuracy with EPSILON={}:".format(EPSILON), accuracy)
    return accuracy

accuracy(val_ds, model, EPSILON)



#Using Mean Poisson deviance
from sklearn.metrics import mean_poisson_deviance

print("Mean Poisson deviance:", mean_poisson_deviance(targets, predictions))




Accuracy with EPSILON=2: 0.620918984280532
Mean Poisson deviance: 6.875492406912289


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)


# We groupby images by ID

In [79]:
def groupbyID(data_joined_test):

  # We add the prediction on the test set to gather our results order by id
  data_joined_test['Prediction'] = predictions

  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)

  temp_dataset = data_joined_test.groupby(['Property Reference Id'])
  tab = temp_dataset.mean()

  data_joined_test.loc[data_joined_test.Prediction<=0,'Prediction']=10**(-0.05)
  print("Final mean Poisson deviance(per ID):", mean_poisson_deviance(data_joined_test["Demand"], data_joined_test["Prediction"]))
  print()
  print()

  return data_joined_test


data_joined_test = groupbyID(data_joined_test)



Final mean Poisson deviance(per ID): 6.875492406912289




<ipython-input-79-a5013f4c0158>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test['Prediction'] = predictions
<ipython-input-79-a5013f4c0158>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)
<ipython-input-79-a5013f4c0158>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_

In [82]:
data_joined_test

,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
136,0200.26.4088,1.086777,5.910983,5.508250,-6.740355,3.543104,-1.425387,2.143826,5.055698,0.219763,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,0.891251
137,0200.26.4088,1.086777,5.910983,5.508250,-6.740355,3.543104,-1.425387,2.143826,5.055698,0.219763,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,1.0,3.069528
138,0200.26.4088,0.747613,8.044648,6.853884,-9.067966,5.848015,-2.940033,2.697528,6.839394,0.334826,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,0.891251
139,0200.26.4088,0.747613,8.044648,6.853884,-9.067966,5.848015,-2.940033,2.697528,6.839394,0.334826,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,1.0,2.510523
140,0214.61.8006,1.744865,10.902982,8.186845,-11.280316,9.826567,-5.122185,4.418775,-1.203919,-2.246455,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,0.891251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8389,9880.50.9682,0.264449,5.243738,5.168600,-6.053524,2.638055,-0.554635,1.017497,2.485184,0.522255,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,3.0,2.805636
8390,9880.61.7822,-0.772212,4.750989,5.042181,-5.838104,2.066901,-1.384502,0.331977,2.852991,1.345560,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,6.0,2.608027
8391,9880.61.7822,-0.634423,6.502183,6.030040,-6.940339,3.809046,-1.138469,0.805016,5.655916,0.790457,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,6.0,2.404353
8392,9880.76.8378,0.264449,5.243738,5.168600,-6.053524,2.638055,-0.554635,1.017497,2.485184,0.522255,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,0.0,1.743060


## 1st version of groupby

In [75]:
# We add the prediction on the test set to gather our results order by id
data_joined_test['Prediction'] = predictions
data_joined_test.sample(5)


<ipython-input-75-b8612112f83e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test['Prediction'] = predictions


,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
6787,7684.61.7705,-0.502878,5.947612,5.516549,-6.756667,3.567677,-2.863783,1.876810,4.323555,1.001609,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,1.0,0.891251
446,0668.61.0065,-0.463543,5.042772,5.045663,-6.140898,2.529379,-2.926522,1.945866,4.458686,1.526753,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,14.0,1.026594
7252,8341.50.7705,-0.955973,4.044521,4.508246,-4.677773,1.006604,0.447340,0.144032,2.436692,1.052145,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,14.0,1.254935
2125,2490.22.0964,-1.118535,4.259557,4.679176,-4.985484,1.229805,0.267612,-0.101243,2.276172,1.229883,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,2.042334
7850,9179.61.4593,-1.852610,5.425081,5.757202,-6.920499,2.561271,-1.741298,-1.045670,-1.061599,2.489897,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,1.0,0.891251


In [42]:
data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)
data_joined_test.dtypes


<ipython-input-42-342d6ca3f86a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)


Property Reference Id     object
0                        float64
1                        float64
2                        float64
3                        float64
                          ...   
Geo_VS                   float64
Geo_ZG                   float64
Geo_ZH                   float64
Demand                   float64
Prediction               float64
Length: 70, dtype: object

In [43]:
temp_dataset = data_joined_test.groupby(['Property Reference Id'])
tab = temp_dataset.mean()
display(tab.sample(5))

<ipython-input-43-36475d7a97ee>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tab = temp_dataset.mean()


,0,1,2,3,4,5,6,7,8,9,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
Property Reference Id,,,,,,,,,,,,,,,,,,,,,
7276.22.0065,-0.151769,4.691804,4.763517,-5.378359,2.089910,-1.122222,0.730841,1.588573,0.597662,5.687003,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,0.000000,0.891251
7977.22.5664,0.213037,4.904241,4.894507,-5.553480,2.216729,-0.663933,0.927539,1.938606,0.325356,5.876572,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.000000,1.670750
5888.22.9935,0.472074,5.079700,4.953190,-5.986607,2.689590,-1.171054,1.182277,2.387276,0.408414,5.914227,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,2.000000,0.891251
6791.61.4266,0.016830,4.677358,4.784645,-5.300134,1.915972,-0.340583,0.822994,2.344957,0.440216,5.762862,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,4.333333,1.232676
9564.26.4593,-0.107945,5.242588,5.112111,-6.010555,2.570060,-0.540652,0.367418,1.305161,0.755894,6.135576,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,8.000000,2.948352


In [44]:
data_joined_test.loc[data_joined_test.Prediction<=0,'Prediction']=10**(-0.05)
print("Final mean Poisson deviance(per ID):", mean_poisson_deviance(data_joined_test["Demand"], data_joined_test["Prediction"]))


Final mean Poisson deviance(per ID): 7.065035586813161


#Testing the overall model on a new testset

In [45]:
def accuracy(y_pred,y,tol=0):
    numpy_y=y.to_numpy()
    rounded_y_pred=y_pred.round()
    assert len(rounded_y_pred)==len(numpy_y)
    return (np.abs(rounded_y_pred-numpy_y)<=tol).sum()/len(rounded_y_pred)


def evaluate_model(y_true,y_pred):
    prediction=y_pred.copy()
    prediction[prediction<=0]=10**(-10)
    return mean_poisson_deviance(y_true,prediction)
    

scoring=make_scorer(evaluate_model,greater_is_better=False)